In [52]:
# set up
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import EarlyStopping

# supress the warnings
import warnings
warnings.filterwarnings("ignore")

In [53]:
# load the dataset
data = pd.read_csv("admissions_data.csv")
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [54]:
# split the dataset into features and labels
features = data.drop(columns=["Chance of Admit "])
labels = data["Chance of Admit "]

# split the dataset into training and test sets
features_training, features_test, labels_training, labels_test = train_test_split(features, labels, test_size=0.25, random_state=42)

# scale the dataset using ColumnTransformer
scaler = StandardScaler()
features_training_scaled = scaler.fit_transform(features_training)
features_test_scaled = scaler.transform(features_test)

In [55]:
# create a neural network model
def dnn_model(neurons=32, learning_rate=0.01):
    model = Sequential()
    # add an input layer
    model.add(InputLayer(input_shape=(features_training.shape[1],)))
    # add a hidden layer of 16 neurons and a ReLU function
    model.add(Dense(neurons, activation='relu'))
    # finally add an output layer
    model.add(Dense(1))
    
    # create an Adam optimizer
    opt = Adam(learning_rate)
    # compile the model
    model.compile(loss="mse", metrics=["mae"], optimizer=opt)
    
    return model

In [56]:
# instantiate the model
model = dnn_model()

# create an early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# fit the model
hist = model.fit(features_training_scaled, labels_training, epochs=50, batch_size=10, verbose=1, validation_split=0.2 , callbacks=[early_stopping])

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3594 - mae: 0.4613 - val_loss: 0.0383 - val_mae: 0.1446
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0319 - mae: 0.1389 - val_loss: 0.0141 - val_mae: 0.0851
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0148 - mae: 0.0976 - val_loss: 0.0106 - val_mae: 0.0790
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0077 - mae: 0.0667 - val_loss: 0.0077 - val_mae: 0.0650
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0083 - mae: 0.0718 - val_loss: 0.0058 - val_mae: 0.0577
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0066 - mae: 0.0636 - val_loss: 0.0079 - val_mae: 0.0704
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0059 - mae: 0.0609 - val_loss: 0.0058 - val_mae: 0.0606
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0052 - mae: 0.0566 - val_loss: 0.0063 - val_mae: 0.0635
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0045 - mae: 

In [59]:
# tune the hyperparameters
# define the parameter grid
param_grid = {
    "batch_size": np.arange(2, 11, 2),
    "epochs": np.arange(10, 50, 15),
    "model__learning_rate": np.logspace(-3, -1, 3),
    "model__neurons": [16 ,32, 64]
}

# perform a grid search
search_model = KerasRegressor(model=dnn_model, verbose=0)
gs = GridSearchCV(search_model, param_grid, scoring="neg_mean_squared_error", cv=3)
grid_result = gs.fit(features_training_scaled, labels_training)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -0.00410442429376832 using {'batch_size': 6, 'epochs': 10, 'model__learning_rate': 0.1, 'model__neurons': 32}
